<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;">Objective</h3>
<br><strong style="color: #000000;"><em>Find and Delete Unattached AWS Elastic IPs</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Release-Unattached-AWS-Elastic-IPs"><strong><u>Release Unattached AWS Elastic IPs</u></strong></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find unattached Elastic IPs</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete unattached Elastic IPs</a></p>

In [25]:
if region == None:
    region = ''
if allocation_ids and not region:
    raise SystemExit("Provide a region for the Allocation ID's!")

<h3 id="Find-unattached-Elastic-IPs"><a id="1" target="_self" rel="nofollow"></a>Find unattached Elastic IPs</h3>
<p>Using unSkript's Find unattached Elastic IPs action, we will find unattahched Elastic IPs which don't have any instances associated to them.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unused_ips</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions


from beartype import beartype
@beartype
def aws_list_unattached_elastic_ips_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def aws_list_unattached_elastic_ips(handle, region: str = "") -> Tuple:
    """aws_list_unattached_elastic_ips Returns an array of unattached elastic IPs.

        :type region: string
        :param region: AWS Region.

        :rtype: Tuple with status result and list of unattached elastic IPs.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            # Filtering the public_ip by region
            ec2Client = handle.client('ec2', region_name=reg)
            all_eips = ec2Client.describe_addresses()
            for eip in all_eips["Addresses"]:
                vpc_data = {}
                if 'AssociationId' not in eip:
                    vpc_data["public_ip"] = eip['PublicIp']
                    vpc_data["allocation_id"] = eip['AllocationId']
                    vpc_data["region"] = reg
                    result.append(vpc_data)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not allocation_ips",
    "condition_result": true
    }''')

task.configure(outputName="unused_ips")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_unattached_elastic_ips, lego_printer=aws_list_unattached_elastic_ips_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-unattached-Elastic-IPs">Create List of unattached Elastic IPs</h3>
<p>This action filters regions that have no unattached Elastic IPs and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_unused_ips</code></p>
</blockquote>

In [28]:
all_unused_ips = []
try:
    if unused_ips[0] == False:
        if len(unused_ips[1])!=0:
            all_unused_ips=unused_ips[1]
except Exception:
    for ids in allocation_ids:
        data_dict = {}
        data_dict["region"] = region
        data_dict["allocation_id"] = ids
        all_unused_ips.append(data_dict)
print(all_unused_ips)
task.configure(outputName="all_unused_ips")

<h3 id="Delete-unattached-Elastic-IPs"><a id="2" target="_self" rel="nofollow"></a>Delete unattached Elastic IPs</h3>
<p>This action deleted unattached Elastic IPs found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>region, elastic_ip</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_release_elastic_ip_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_release_elastic_ip(handle, region: str, allocation_id: str) -> Dict:
    """aws_release_elastic_ip release elastic ip.

        :type allocation_id: string
        :param allocation_id: Allocation ID of the Elastic IP to release.

        :type region: string
        :param region: AWS Region.

        :rtype: Dict with the release elastic ip info.
    """
    try:
        ec2_Client = handle.client('ec2', region_name=region)
        response = ec2_Client.release_address(AllocationId=allocation_id)
        return response
    except Exception as e:
        raise Exception(e)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "allocation_id": "iter.get(\\"allocation_id\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_unused_ips",
    "iter_parameter": ["allocation_id","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_unused_ips)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_release_elastic_ip, lego_printer=aws_release_elastic_ip_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;">Conclusion</h3>
<p>In this Runbook, we were able to check for any unattached Elastic IP (EIP) addresses in our AWS account and release (remove) them in order to lower the cost of your monthly AWS bill. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>